In [2]:
from __future__ import division
from collections import defaultdict

%matplotlib inline
import numpy as np
import pandas as pd
import thinkdsp
import thinkplot

In [3]:
#Ryan's fancy way of inputting data in an easier way
data_dict = {'walking':{},'jogging':{},'upstairs':{},'downstairs':{}}
names = ['meg','ryan','dennis']
acts = ['walking', 'jogging', 'upstairs', 'downstairs']
for name in names:
    data_file_names = ['data/{}_{}_long.csv'.format(name, activity) for activity in acts]
    for i,file in enumerate(data_file_names):
        df = pd.read_csv(file)
        data_dict[acts[i]][name] = df

In [4]:
db = {}

for i, (g, gdf) in enumerate(df.groupby('personID')):
    db[g] = gdf

KeyError: 'personID'

In [ ]:
print "PERSON {}".format(33)
print db[33].head()

##Pipeline...

###Extract Features

In [5]:
from dominant_frequency import find_dominant

In [6]:
for personID, df in db.iteritems():
    
    break

In [7]:
def extract_features(vals, framerate, dom_freq_method="autocorr"):
    """
    Arguments
    ---------
    vals: array-like, shape (n_samples,)
        evenly spaced in time values, (i.e. magnitude of the accleration vector)
        
    dom_freq_method: string, default, "autocorr"
        available strings:
        "autocorr" - calculated using autocorrelation
        "spectrum" - calculated using peaks method of frequency spectrum

    Returns
    -------
    obs: array-like, shape (n_windows, n_features)
        current features: amp_dist, dom_freq
    """
    # dictionary of arrays, where the arrays will be a list of amplitudes for each activity
    amp_dist = defaultdict(list)
    domfreq_dist = defaultdict(list)
    domfreq_dist2 = defaultdict(list)

    # Groupby the 4 Activites
    for g, gdb in db.groupby('activity'):
        # Calculate Waves
        # FIX: framerate is probably not calculated correctly
        framerate = 100000
        zwave = thinkdsp.Wave(vals, framerate=framerate)

        zwave
        start0 = 0
        window_size = 0.0003
        step_size = window_size / 2
        seg_nums = 120

        for i in range(seg_nums):
            zseg = zwave.segment(start=start0+i*step_size, duration=window_size)

            # Unbiasing the Wave to make Spectrum amplitude at frequency 0, equal to 0
            zseg.unbias()

            # Convert to Frequency Domain
            spectrum = zseg.make_spectrum()

            # Extract Peaks
            # peak[0] the highest peak
            peaks = spectrum.peaks()

            # appending the amplitude of the highest peak to our dictionary of amplitudes for each activity

            # peak[0][0] is the amplitude of the highest peak
            # peak[0][1] is the frequency of the highest peak
            amp_dist[g].append(peaks[0][0])
            domfreq_dist2[g].append(peaks[0][1])

            # dominant frequency
            domfreq_dist[g].append(find_dominant(zseg))

Just a check, to see the mean and standard deviation that characterize the distribution of peak amplitude values

In [8]:
for (activity, amps), (activity, domfreqs), (activity, domfreqs2) in zip(amp_dist.iteritems(), domfreq_dist.iteritems(), domfreq_dist2.iteritems()):
    print activity
    print "mean amp: ",np.array(amps).mean()
    print "std amp: ",np.array(amps).std()
    print "mean domfreq: ", np.array(domfreqs).mean()
    print "std domfreq: ", np.array(domfreqs).std()
    print "mean domfreq2: ", np.array(domfreqs2).mean()
    print "std domfreq2: ", np.array(domfreqs2).std()

NameError: name 'amp_dist' is not defined

###Learn 

In [33]:
from hmmlearn.hmm import GaussianHMM

# dict of gaussian hidden markov models for each activity
ghmm = {}

# dict of train data for each activity
X_train = {}

# dict of test data for each activity
X_test = {}

# number of hidden states
n_components = 3

# number of samples in the training set
train_size = 60

# Train a separate GHMM for each activity
for activity, amps in amp_dist.iteritems():
    # Create GHMM
    ghmm[activity] = GaussianHMM(n_components, covariance_type="diag", n_iter=1000)
    
    # Split into Train and Test Data (No Random Shuffling Now)
    # If we wanted to add more features:
    # X_train[activity] = np.column_stack([amp_dist[activity][:train_size], next_feature[activity][:train_size])
    
    features = (amp_dist, domfreq_dist2)

    X_train[activity] = np.column_stack([feature[activity][:train_size] for feature in features])
    X_test[activity] = np.column_stack([feature[activity][train_size:] for feature in features])
    
    # Fit on Training Data
    # Confused about .fit([X])
    ghmm[activity].fit([X_train[activity]])

###Predict

In [1]:
X

NameError: name 'X' is not defined

In [34]:
activities = ["Jogging", "Walking", "Upstairs", "Downstairs"]

# For each Test Set
for activity, X in X_test.iteritems():
    print "actual activity: " + activity
    
    # logprobs for each activity_model
    # the log-likelihood that the given sequence of observations looks like things this model could produce
    logprobs = {}

    # Try Out the 4 models
    for model_activity, model in ghmm.iteritems():
        # model.score returns log likelihood of the observation
        logprobs[model_activity] = model.score(X)
    

    # Which ever has the highest probability will be the model
    max_idx = np.argmax(np.array([logprobs[activity] for activity in activities]))
    print max_idx
    pred_activity = activities[max_idx]
    print "predicted activity: " + pred_activity
    
    print "logprobs: "
    print logprobs
    
    print "\n"


actual activity: Walking
0
predicted activity: Jogging
logprobs: 
{'Walking': -908.01678777656798, 'Downstairs': -2835.7986967145257, 'Jogging': -907.94465196170586, 'Upstairs': -1349.7659876207963}


actual activity: Downstairs
3
predicted activity: Downstairs
logprobs: 
{'Walking': -1020.1666103737786, 'Downstairs': -776.81482303212636, 'Jogging': -1024.372651773861, 'Upstairs': -1190.9363556798253}


actual activity: Jogging
0
predicted activity: Jogging
logprobs: 
{'Walking': -1058.3666544800637, 'Downstairs': -12461.628264153218, 'Jogging': -894.18084333649404, 'Upstairs': -4610.4205010823407}


actual activity: Upstairs
2
predicted activity: Upstairs
logprobs: 
{'Walking': -1104.1912395135478, 'Downstairs': -1497.17352096471, 'Jogging': -972.90500722202, 'Upstairs': -394.13945927885669}




- ```n_components``` makes a difference, for classifying between Upstairs and Downstairs.  2 and 4 seem to misclassify, 3 and 5 classify accurately.  
- note that the dictionary keys will not be in the same order as the ```activities``` list


##TODOS:

- phone accelerometer
    - collect our own dataset
- timeseries spacing
    - can interpolate given the timestamps
- feature visualization / extraction
    - frequency bins
    - n most dominant frequencies
- machine learning pipeline building
    - Investigating Assumptions of Training / Evaluation Procedure
    - Visualizations of labels versus probability classifications
    - Randomized Train Test Split
    - Cross Validation (how to determine what's the best ```n_components```
    
##DONE:

- phone accelerometer
    - install app
- feature visualization / extraction
    - timeseries
    - frequency spectrum
    - power spectrum
    - phase
    - dominant frequency by autocorrelation
- machine learning pipeline building
    - Hidden Markov Model
